In [0]:
def create_user(display_name,email):
  import requests
  import json
  headers = {"Authorization": "Bearer " + dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()}
  base_url = "https://" + dbutils.notebook.entry_point.getDbutils().notebook().getContext().browserHostName().get()
  payload = {
        "schemas": ["urn:ietf:params:scim:schemas:core:2.0:User"],
        "userName": email,
        "displayName": display_name,
        "active": True
  }
  endpoint = f'/api/2.0/preview/scim/v2/Users'

  # Make the API request
  response = requests.post(base_url + endpoint, headers = headers, data=json.dumps(payload)).json()
  return response

In [0]:
def add_to_group(group_id,user_id):
  import requests
  import json
  headers = {"Authorization": "Bearer " + dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()}
  base_url = "https://" + dbutils.notebook.entry_point.getDbutils().notebook().getContext().browserHostName().get()
  payload = {
    "schemas": ["urn:ietf:params:scim:api:messages:2.0:PatchOp"],
    "Operations": [
        {
            "op": "add",
            "path": "members",
            "value": [{"value": user_id}]
        }
    ]
}
  endpoint = f'/api/2.0/preview/scim/v2/Groups/{group_id}'

  # Make the API request
  response = requests.patch(base_url + endpoint, headers = headers, data=json.dumps(payload)).json()
  return response

In [0]:
import os
from pyspark.sql import DataFrame
id_do_grupo="COLOCAR_AQUI_O_ID_DO_GRUPO"
# Get the list of files in the directory
directory = "/Volumes/jsf_catalog/ecommerce/jsf_vol_ecommerce"
file = os.listdir(directory)
# Create a DataFrame from the file
df = spark.read.format("csv").option("header", "true").load(os.path.join(directory, file))
for rows in df.collect():
  email=rows['email_aluno']
  user_name=rows['nome_aluno']
  new_user=create_user("nome_aluno","email_aluno")
  user_id=new_user['id']
  group_id=id_do_grupo
  add_to_group(group_id,user_id)